# **Can you escape a maze without walls?**
## [Riddler Express, Feb 1, 2019](https://fivethirtyeight.com/features/can-you-escape-a-maze-without-walls/)

### solution by [Laurent Lessard](https://laurentlessard.com)

In [1]:
import numpy as np
import pandas as pd

# Here is what the (fixed) grid looks like (padded using "X")
G = ( ('X','X','X','X','X','X','X','X','X','X'),
      ('X','L','U','U','?','U','L','X','L','X'),
      ('X','R','L','R','L','U','🙂','U','U','X'),
      ('X','S','L','R','L','U','L','X','R','X'),
      ('X','U','R','?','R','S','L','?','R','X'),
      ('X','R','U','U','R','R','R','S','L','X'),
      ('X','S','?','S','L','S','S','L','R','X'),
      ('X','R','L','R','?','R','L','?','L','X'),
      ('X','L','R','S','R','S','L','R','L','X'),
      ('X','X','X','X','X','X','X','X','X','X') )

# we'll call a STATE a tuple (x,y,d) corresponding to:
# x: the first coordinate (row number) on the grid
# y: the second coordinate (column number) on the grid
# p: the direction we just moved in (0,1,2,3), corresponding to (right,up,left,down)

# given that we are in a certain state, what is the corresponding letter on the grid?
def get_symbol( state ):
    return G[state[0]][state[1]]

# given we are in a certain state, return the state we land in if we move in a given direction
def state_update( state, move_direction ):
    x,y,t = state
    d = move_direction % 4
    if d == 0:
        return (x,y+1,d)
    elif d == 1:
        return (x-1,y,d)
    elif d == 2:
        return (x,y-1,d)
    else: # d == 3
        return (x+1,y,d)

# what are all the possible next moves from a given state?
def next_possible_directions( state ):
    d = state[2]
    s = get_symbol( state )
    if s == 'S':
        return [d]
    elif s == 'L':
        return [(d+1) % 4]
    elif s == 'R':
        return [(d-1) % 4]
    elif s == 'U':
        return [(d+2) % 4]
    elif s == '?':
        return range(4)
    print('ERROR: accessing next move of winning state')

def get_cost( state ):
   
    # if we have already successfully computed the cost for this state, return its cost
    ctg = cost_to_go.get(state, None)
    if ctg != None:
        return ctg
    else:
        # we've been here before (and didn't compute the cost), must be infinite loop. Get out!
        if state in visited:
            ctg = np.inf
        else:
            # terminal conditions (either we win or it's a dead end)
            s = get_symbol(state)
            if s == '🙂':
                solutions[state] = [state]
                ctg = 1
            elif s == 'X':
                ctg = np.inf
            else:
                visited.add(state) # make sure we only recurse once per state
                next_states = [ state_update(state,direction) for direction in next_possible_directions(state) ]
                costs = [ get_cost(next_state) for next_state in next_states ]
                ctg = 1 + min( costs, default = np.inf )
                if ctg != np.inf:
                    ixmin = np.argmin(costs)
                    solutions[state] = [state] + solutions[next_states[ixmin]]
            cost_to_go[ state ] = ctg
        return ctg

In [3]:
cost_to_go = dict()  # record the cost to go
solutions = dict()   # record all the solutions
visited = set()      # make sure we have no infinite loops

# possible starting states
start_states = ( [(k,1,0) for k in range(1,9)] + [(8,k,1) for k in range(1,9)] + 
                 [(k,8,2) for k in range(1,9)] + [(1,k,3) for k in range(1,9)] )

# associated costs-to-go
CTG = [ get_cost(state) for state in start_states ]

# put the answers in the grid!
Gsol = pd.DataFrame(np.array(G))
for (i,state) in enumerate(start_states):
    p = state_update(state, state[2]+2)
    Gsol.iloc[p[0],p[1]] = CTG[i]

Gsol

,0,1,2,3,4,5,6,7,8,9
0,X,inf,inf,inf,34,inf,inf,inf,inf,X
1,inf,L,U,U,?,U,L,X,L,inf
2,42,R,L,R,L,U,🙂,U,U,inf
3,inf,S,L,R,L,U,L,X,R,inf
4,inf,U,R,?,R,S,L,?,R,inf
5,inf,R,U,U,R,R,R,S,L,inf
6,inf,S,?,S,L,S,S,L,R,45
7,inf,R,L,R,?,R,L,?,L,inf
8,inf,L,R,S,R,S,L,R,L,inf
9,X,inf,inf,inf,inf,inf,inf,inf,inf,X


In [6]:
# print the sequence of moves that gets you to the smiley face from starting point (1,4) moving down (direction 3)
sol = solutions[(1,4,3)]
for state in sol:
    print( (state[0],state[1],['right','up   ','left ','down '][state[2]]), get_symbol(state) )

(1, 4, 'down ') ?
(2, 4, 'down ') L
(2, 5, 'right') U
(2, 4, 'left ') L
(3, 4, 'down ') L
(3, 5, 'right') U
(3, 4, 'left ') L
(4, 4, 'down ') R
(4, 3, 'left ') ?
(4, 4, 'right') R
(5, 4, 'down ') R
(5, 3, 'left ') U
(5, 4, 'right') R
(6, 4, 'down ') L
(6, 5, 'right') S
(6, 6, 'right') S
(6, 7, 'right') L
(5, 7, 'up   ') S
(4, 7, 'up   ') ?
(4, 6, 'left ') L
(5, 6, 'down ') R
(5, 5, 'left ') R
(4, 5, 'up   ') S
(3, 5, 'up   ') U
(4, 5, 'down ') S
(5, 5, 'down ') R
(5, 4, 'left ') R
(4, 4, 'up   ') R
(4, 5, 'right') S
(4, 6, 'right') L
(3, 6, 'up   ') L
(3, 5, 'left ') U
(3, 6, 'right') L
(2, 6, 'up   ') 🙂
